In [ ]:
#|hide
#|default_exp processors

# nbprocess.processors
- Some processors for `NBProcessor`

In [ ]:
#|export
from nbprocess.read import *
from nbprocess.imports import *
from nbprocess.process import *

from fastcore.imports import *
from fastcore.xtras import *

In [ ]:
from fastcore.test import *

In [ ]:
_test_file = '../tests/docs_test.ipynb'

def _run_procs(procs=None, preprocs=None, postprocs=None):
    nbp = NBProcessor(_test_file, procs, preprocs=preprocs, postprocs=postprocs)
    nbp.process()
    return '\n'.join([cell.source for cell in nbp.nb.cells])

### Injecting Metadata Into Cells -

In [ ]:
#|export
_re_meta= r'^\s*#(?:cell_meta|meta):\S+\s*[\n\r]'

def inject_meta(cell):
    "Inject metadata into a cell for further preprocessing with a comment."
    _pattern = r'(^\s*#(?:cell_meta|meta):)(\S+)(\s*[\n\r])'
    if cell.cell_type == 'code' and re.search(_re_meta, cell.source, flags=re.MULTILINE):
        cell_meta = re.findall(_pattern, cell.source, re.MULTILINE)
        d = cell.metadata.get('nbprocess', {})
        for _, m, _ in cell_meta:
            if '=' in m:
                k,v = m.split('=')
                d[k] = v
            else: print(f"Warning cell_meta:{m} does not have '=' will be ignored.")
        cell.metadata['nbprocess'] = d

In [ ]:
#|export
def show_meta(cell):
    "Show cell metadata"
    meta = cell.metadata.get('nbprocess')
    if meta: print(meta)

To inject metadata make a comment in a cell with the following pattern: `#cell_meta:{key=value}`. Note that `#meta` is an alias for `#cell_meta`

For example, at the moment, this notebook has no cells with metadata, which we can see b using `show_meta`:

In [ ]:
_run_procs([show_meta]);

However, after we process this notebook with `inject_meta`, the appropriate metadata will be injected:

In [ ]:
_run_procs([inject_meta,show_meta]);

### Insert Warning Into Markdown -

In [ ]:
#| export
def insert_warning(nb):
    "Insert Autogenerated Warning Into Notebook after the first cell."
    content = "<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->"
    nb.cells.insert(1, AttrDict(cell_type='markdown', metadata={}, source=content))

This preprocessor inserts a warning in the markdown destination that the file is autogenerated.  This warning is inserted in the second cell so we do not interfere with front matter.

In [ ]:
res = _run_procs(preprocs=[insert_warning])
assert "<!-- WARNING: THIS FILE WAS AUTOGENERATED!" in res

In [ ]:
 _run_procs([inject_meta, show_meta]);

### Remove cell bits based on tags

In [ ]:
#|export
def update_tags(cell):
    root = cell.metadata.get('nbprocess', {})
    tags = root.get('tags', root.get('tag')) # allow the singular also
    if tags: cell.metadata['tags'] = cell.metadata.get('tags', []) + tags.split(',')

`update_tags` is meant to be used with `inject_meta` to configure the visibility of cells in rendered docs.

In [ ]:
def TagRemove(cell_tag=None, outputs_tag=None, input_tag=None):
    def _inner(cell):
        tags = nested_idx(cell, 'metadata', 'tags') or []
        if cell_tag in tags: cell['source'] = None
        if outputs_tag in tags and 'outputs' in cell: del(cell['outputs'])
        if input_tag in tags: del(cell['source'])
    return _inner

In [ ]:
res = _run_procs([inject_meta, update_tags, TagRemove("remove_cell",'remove_output','remove_input')])
assert 'you will not be able to see this cell at all either' not in res

AssertionError: 

## Export -

In [ ]:
#|skip
basic_export_nb2('00_read.ipynb', 'read')
basic_export_nb2('01_maker.ipynb', 'maker')
basic_export_nb2('02_process.ipynb', 'process')

g = exec_new('import nbprocess.process')
assert hasattr(g['nbprocess'].process, 'NBProcessor')